In [ ]:
using Flux
using Distributions
using Flux, Flux.Data.MNIST, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated
# using CUDAapi

In [ ]:
# if has_cuda()
#     @info "CUDA is on"
#     import CuArrays
#     CuArrays.allowscalar(false)
priors=zeros(Float64,(3,500))
size(priors)

In [ ]:
## Generate data

priors=zeros(Float64,(3,500))

## this is an arbitrary OED estimator
function fake_OED(prior)
    p1,p2,p3=prior
    max_val=max(p1,p2,p3)
    if max_val==p1
        return 7
    elseif max_val==p2
        return 0
    else
        return 5
    end
end

In [ ]:
for idx in 1:500
    first_prior=rand(Uniform(0.0,0.99),1)[1]
    second_prior=rand(Uniform(0.0,1-first_prior),1)[1]
    third_prior=1.0-first_prior-second_prior
    prior=[first_prior,second_prior,third_prior]
    priors[:,idx]=prior
end


In [ ]:
## Create mapping from integer to experiment
int_to_experiment=Dict(
    1 => "HHHH", 
    2 => "HHHT", 
    3 => "HHTH",
    4 => "HTHH",
    5 => "THHH",
    6 => "HHTT",
    7 => "TTHH",
    8 => "HTHT",
    9 => "THTH",
    10 => "HTTH",
    11 => "THHT",
    12 => "TTTH",
    13 => "TTHT",
    14 => "THTT",
    15 => "HTTT",
    16 => "TTTT"
)


In [ ]:
size(labels)

In [ ]:
labels=Any[]
for prior in eachcol(priors)
    push!(labels,fake_OED(prior))
end

In [ ]:
model=Chain(
Dense(3,16,relu),
Dense(16,32,relu),
Dense(32,16,relu),
    softmax)#|> gpu




In [ ]:
loss(x, y) = crossentropy(model(x), y)
accuracy(x, y) = mean(onecold(cpu(model(x))) .== onecold(cpu(y)))

In [ ]:
Y=onehotbatch(labels,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])|> gpu
X=priors #|> gpu

dataset=repeated((X[:,0:400],Y[:,0:400]),500)
evalcb=() -> @show (loss(X,Y))
opt=ADAM()


In [ ]:
ps=Flux.params(model)
Flux.train!(loss, ps, dataset, opt, cb = throttle(evalcb, 10))


In [ ]:
@show accuracy(X, Y)

In [ ]:
size(X[:,1--400])

In [ ]:
function generate_data(n_samples,epochs)
for idx in 1:n_samples
    first_prior=rand(Uniform(0.0,0.99),1)[1]
    second_prior=rand(Uniform(0.0,1-first_prior),1)[1]
    third_prior=1.0-first_prior-second_prior
    prior=[first_prior,second_prior,third_prior]
    priors[:,idx]=prior
end
labels=Any[]
for prior in eachcol(priors)
    push!(labels,fake_OED(prior))
end
Y=onehotbatch(labels,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])|> gpu
X=priors |> gpu
return repeated((X,Y),epochs),X,Y
end
data,t1,t2=generate_data(500,1)

In [ ]:
t1


In [10]:
using Flux
using Distributions
using Flux, Flux.Data.MNIST, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated

function fake_OED(prior)
    p1,p2,p3=prior
    max_val=max(p1,p2,p3)
    if max_val==p1
        return 7
    elseif max_val==p2
        return 0
    else
        return 5
    end
end

##mapping from int to sequence 
int_to_experiment=Dict(
    1 => "HHHH", 
    2 => "HHHT", 
    3 => "HHTH",
    4 => "HTHH",
    5 => "THHH",
    6 => "HHTT",
    7 => "TTHH",
    8 => "HTHT",
    9 => "THTH",
    10 => "HTTH",
    11 => "THHT",
    12 => "TTTH",
    13 => "TTHT",
    14 => "THTT",
    15 => "HTTT",
    16 => "TTTT"
)

##Generate fake priors, in the future learn these with a NN\
function generate_data(n_samples,epochs)
    priors=zeros(Float64,(3,n_samples))
    for idx in 1:n_samples
        first_prior=rand(Uniform(0.0,0.99),1)[1]
        second_prior=rand(Uniform(0.0,1-first_prior),1)[1]
        third_prior=1.0-first_prior-second_prior
        prior=[first_prior,second_prior,third_prior]
        priors[:,idx]=prior
    end
    labels=Any[]
    for prior in eachcol(priors)
        push!(labels,fake_OED(prior))
    end
    Y=onehotbatch(labels,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])|> gpu
    X=priors |> gpu
    return repeated((X,Y),epochs),X,Y
end

model=Chain(
Dense(3,16,relu),
Dense(16,32,relu),
Dense(32,16,relu),
    softmax)|> gpu

loss(x, y) = crossentropy(model(x), y)
accuracy(x, y) = mean(onecold(cpu(model(x))) .== onecold(cpu(y)))

train_dataset,train_x,train_y=generate_data(8000,500)
test_dataset,test_x,test_y=generate_data(100,1)
evalcb=() -> @show (loss(train_x,train_y))
opt=ADAM()

ps=Flux.params(model)
Flux.train!(loss, ps, train_dataset, opt, cb = throttle(evalcb, 10))
println("Training accuracy:")
@show accuracy(train_x, train_y)
println("Testing accuracy:")
@show accuracy(test_x, test_y)

loss(train_x, train_y) = 2.8001204f0
loss(train_x, train_y) = 0.7482724f0
Training accuracy:
accuracy(train_x, train_y) = 0.78625
Testing accuracy:
accuracy(test_x, test_y) = 0.76


0.76